In [1]:
import os
import argparse
import speech_recognition as sr 
from pydub import AudioSegment 
from pydub.silence import split_on_silence
import errno
from pydub.playback import play
from pydub.silence import detect_silence

filename = raw_input("Enter video file name : ") 

Enter video file name : SVM_MARGIN


In [2]:
file_list = os.listdir("./Audio/")
audiofilename = filename + '.wav'  #with extension
if audiofilename in file_list:
    print(".wav already exists")
else:
    os.system("ffmpeg -i Video/"+ filename + ".mp4 Audio/" + filename + ".wav")
    print(".mp4 file converted to .wav file:",audiofilename)  

.wav already exists


In [3]:
path = "Audio/"+audiofilename
sound = AudioSegment.from_wav(path) 
print('Loudness of audio',sound.dBFS)

('Loudness of audio', -16.755035069704146)


In [4]:
try: 
    os.mkdir('Audio_chunks/'+ filename) 
except OSError as e:
    if e.errno == errno.EEXIST:
        print('File exist already.')
    else:
        raise

File exist already.


In [5]:
from pydub.silence import detect_silence
silence_segments = detect_silence(sound, min_silence_len=2000, silence_thresh=-20, seek_step=1)

In [6]:
silence_segments_14_3 = detect_silence(sound, min_silence_len=3000, silence_thresh=-14, seek_step=1)

In [7]:
silence_segments_16_3 = detect_silence(sound, min_silence_len=3000, silence_thresh=-16, seek_step=1)

In [8]:
silence_segments_18_3 = detect_silence(sound, min_silence_len=3000, silence_thresh=-18, seek_step=1)

In [9]:
silence_segments_20_3 = detect_silence(sound, min_silence_len=3000, silence_thresh=-20, seek_step=1)

In [10]:
silence_segments_22_3 = detect_silence(sound, min_silence_len=3000, silence_thresh=-22, seek_step=1)

In [11]:
silence_segments_24_3 = detect_silence(sound, min_silence_len=3000, silence_thresh=-14, seek_step=1)

In [12]:
silence_segments_18_35 = detect_silence(sound, min_silence_len=3500, silence_thresh=-18, seek_step=1)

In [13]:
silence_segments_20_35 = detect_silence(sound, min_silence_len=3500, silence_thresh=-20, seek_step=1)

In [17]:
print('no of segments:',len(silence_segments_20_3)+1)
for segment in silence_segments_18_3:
    check = sound[segment[0]:segment[1]]
    print(check.dBFS)
    print((segment[0]/3600)/1000,(segment[0]/60000)%60,(segment[0]/1000)%60,'-',(segment[1]/3600000),(segment[1]/60000)%60,(segment[1]/1000)%60)

('no of segments:', 90)
-20.3180061811
(0, 0, 0, '-', 0, 0, 20)
-17.4854677669
(0, 0, 21, '-', 0, 0, 25)
-17.9062775997
(0, 0, 28, '-', 0, 0, 32)
-18.6654579454
(0, 0, 32, '-', 0, 0, 45)
-19.5074169231
(0, 0, 50, '-', 0, 1, 2)
-18.185672407
(0, 1, 16, '-', 0, 1, 19)
-18.2266385754
(0, 1, 20, '-', 0, 1, 27)
-17.866791492
(0, 1, 29, '-', 0, 1, 34)
-18.9496605627
(0, 1, 34, '-', 0, 2, 5)
-18.5840059219
(0, 2, 14, '-', 0, 2, 20)
-17.4557520704
(0, 2, 22, '-', 0, 2, 27)
-18.1534666244
(0, 2, 28, '-', 0, 2, 34)
-21.1513607645
(0, 2, 34, '-', 0, 2, 47)
-17.7801381926
(0, 2, 48, '-', 0, 2, 57)
-17.5612056966
(0, 2, 59, '-', 0, 3, 13)
-17.6053577261
(0, 3, 16, '-', 0, 3, 24)
-18.2158392582
(0, 3, 25, '-', 0, 3, 32)
-17.619453296
(0, 3, 33, '-', 0, 3, 37)
-18.304792571
(0, 3, 37, '-', 0, 3, 44)
-17.1927888093
(0, 3, 44, '-', 0, 3, 49)
-18.3858570809
(0, 3, 57, '-', 0, 4, 1)
-18.3026121042
(0, 4, 12, '-', 0, 4, 22)
-16.3295986125
(0, 4, 27, '-', 0, 4, 30)
-17.6780941425
(0, 4, 32, '-', 0, 4, 39)


In [ ]:
# for chunk in chunks:
#     play(chunk)
#     print(len(chunk))
silence_segments = silence_segments_32_3
chunks = []
previous_segment_begin = 0
i=1
for segment in silence_segments:
    if i==1:
        chunk = []
        chunk.append(0)
        chunk.append((segment[1]+segment[0])/2)
        chunk.append(sound[0:(segment[1]+segment[0])/2])
        chunks.append(chunk)
    else:
        chunk = []
        chunk.append(previous_segment_begin)
        chunk.append((segment[1]+segment[0])/2)     
        chunk.append(sound[previous_segment_begin:(segment[1]+segment[0])/2])
        chunks.append(chunk)
    if i == len(silence_segments): 
        chunk = []
        chunk.append((segment[1]+segment[0])/2)
        chunk.append(len(sound)) 
        chunk.append(sound[(segment[1]+segment[0])/2:len(sound)])
        chunks.append(chunk)
    previous_segment_begin = (segment[0]+segment[1])/2
    i = i+1


In [ ]:
print('no of chunks:',len(chunks))
size = 0
for chunk in chunks:
    size += len(chunk[2])
print(size/60000)

In [ ]:
text_data = []
i = 0
for chunk in chunks:
    text_segment = []  # text segment = [start time of segment,end time of segment,text in segment]
    text_segment.append(chunk[0])
    text_segment.append(chunk[1])
    
    chunk_silent = AudioSegment.silent(duration = 100)  
    audio_chunk = chunk_silent + chunk[2] + chunk_silent 
    
    # save chunk in Audio_chunks/filename/
    filepath = "./Audio_chunks/{0}/chunk{1}.wav".format(filename,i)
    audio_chunk.export(filepath, bitrate ='192k', format ="wav") 
    
    # convert speech to text
    r = sr.Recognizer()  
    with sr.AudioFile(filepath) as source: 
        # r.adjust_for_ambient_noise(source) 
        audio_listened = r.listen(source) 
    
    try: 
        rec = r.recognize_google(audio_listened) 
        print(rec)
        text_segment.append(rec)
    except sr.UnknownValueError: 
        print("Could not understand audio") 
    except sr.RequestError as e: 
        print("Could not request results. check your internet connection") 

    text_data.append(text_segment)
    i = i + 1

In [ ]:
for text_segment in text_data:
    if len(text_segment) == 3:
        print((text_segment[0]/1000)/60,(text_segment[0]/1000)%60,'-',(text_segment[1]/1000)/60,(text_segment[1]/1000)%60)
        print(text_segment[2])

In [ ]:
indicator_list = ['equal to','minus',' x ',' y ',' z ',' X ',' Y ',' Z ']
# strength_list = ['function',' x ',' X ',' y ',' Y ',' z ',' Z ','beta','alpha']
strength_list = []
function_segments = []
for text_segment in text_data:
    indicator_count = 0
    strengthener_count = 0
    if len(text_segment) == 3:
        for indicator in indicator_list:
                indicator_count = indicator_count + text_segment[2].count(indicator)
        for strengthener in strength_list:
            strengthener_count = strengthener_count + text_segment[2].count(strengthener)
    if indicator_count >= 1 or strengthener_count >= 3:
        print((text_segment[0]/1000)/60,(text_segment[0]/1000)%60,'-',(text_segment[1]/1000)/60,(text_segment[1]/1000)%60)
        print(text_segment[2])
        function_segments.append(text_segment)
    

In [ ]:
for segment in function_segments:
    print((segment[0]/1000)/60,(segment[0]/1000)%60,'-',(segment[1]/1000)/60,(segment[1]/1000)%60)

In [ ]:
print(len(sound)/60000)